In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(r"train.csv")

# Keep only needed columns
df = df[["image_link", "price"]]

# Drop missing data if any
df = df.dropna(subset=["image_link", "price"])

# Select only first 1000 rows (or sample randomly)
df = df.head(1000)
# OR random sample:
# df = df.sample(n=1000, random_state=42).reset_index(drop=True)

print(df.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [3]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")


NameError: name 'df' is not defined

In [ ]:
import tensorflow as tf
import requests

IMG_SIZE = (224, 224)

def fetch_image_from_url(url):
    """Fetch and preprocess an image from a URL."""
    try:
        # Convert TF string to normal Python string
        url = url.numpy().decode("utf-8")

        response = requests.get(url, timeout=10)
        image = tf.image.decode_jpeg(response.content, channels=3)
        image = tf.image.resize(image, IMG_SIZE)
        image = image / 255.0
    except Exception:
        # Return a blank image if loading fails
        image = tf.ones((*IMG_SIZE, 3))
    return image

def load_data(image_url, price):
    """Wrapper for TensorFlow dataset mapping."""
    image = tf.py_function(func=fetch_image_from_url, inp=[image_url], Tout=tf.float32)
    image.set_shape((*IMG_SIZE, 3))
    return image, tf.cast(price, tf.float32)


In [ ]:
# Convert pandas columns to TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_df["image_link"].values, train_df["price"].values))
val_ds = tf.data.Dataset.from_tensor_slices((val_df["image_link"].values, val_df["price"].values))

# Apply the preprocessing function
train_ds = train_ds.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)

# Batch and prefetch for performance
train_ds = train_ds.batch(16).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(16).prefetch(tf.data.AUTOTUNE)


In [4]:
# Inspect one batch to verify everything works
for images, prices in train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("First few prices:", prices[:5].numpy())


NameError: name 'train_ds' is not defined

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import random

# Pick one random sample from the training dataset
sample_idx = random.randint(0, len(train_df) - 1)
sample_url = train_df.iloc[sample_idx]["image_link"]
sample_price = train_df.iloc[sample_idx]["price"]

print(f"Sample index: {sample_idx}")
print(f"Image URL: {sample_url}")
print(f"Target price: {sample_price}")

# Fetch and display the image manually
import requests
from PIL import Image
from io import BytesIO

try:
    response = requests.get(sample_url, timeout=10)
    image = Image.open(BytesIO(response.content)).convert("RGB")
except Exception as e:
    print("Error loading image:", e)
    image = Image.new("RGB", (224, 224), (255, 255, 255))

plt.imshow(np.array(image))
plt.axis("off")
plt.title(f"Price: {sample_price}")
plt.show()


NameError: name 'train_df' is not defined

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50

# Pretrained backbone (feature extractor)
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base layers (optional: can unfreeze later for fine-tuning)
base_model.trainable = False

# Build model with additional dense layers, dropout, and regularization
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),

    # First dense layer
    layers.Dense(256, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.4),

    # Second dense layer (extra one you asked for)
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),

    # Output layer for regression
    layers.Dense(1)
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,145,281 (92.11 MB)

 Trainable params: 557,569 (2.13 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='mse',
    metrics=['mae']
)


In [21]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,             # stop after 3 epochs of no improvement
    restore_best_weights=True
)


In [1]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stop],
    verbose=1
)


NameError: name 'model' is not defined

In [ ]:
results = model.evaluate(val_ds)
print(f"Validation Loss (MSE): {results[0]:.4f}")
print(f"Validation MAE: {results[1]:.4f}")
